# Multi-agent Collaboration - News Writer
When you need more than a single agent to handle a complex task, you can create additional specialized agents to address different aspects of the process. However, managing these agents becomes technically challenging as tasks grow in complexity. As a developer using open source solutions, you may find yourself navigating the complexities of agent orchestration, session handling, memory management, and other technical aspects that require manual implementation.

With the fully managed multi-agent collaboration capability on Amazon Bedrock, specialized agents work within their domain of expertise, coordinated by a supervisor agent. The supervisor breaks down requests, delegates tasks, and consolidates outputs into a final response. For example, an investment advisory multi-agent system might include agents specialized in financial data analysis, research, forecasting, and investment recommendations. Similarly, a retail operations multi-agent system could handle demand forecasting, inventory allocation, supply chain coordination, and pricing optimization.

In this lab, we will build a multi-agent system where facts about a news event collected by a journalist are used to generate a news story. As shown in the diagram below, multiple agents will be responsible for tasks which are orchestrated by the supervisor agent:

<img src="../../imgs/lab7-architecture-diagram.png" width="800">

The workflow shown in the diagram above is as follows:

1. A journalist submits facts to a front-end backed by an LLM (Interface Supervisor)
2. The Interface Supervisor agent sends the facts to a Research agent.
3. The Research agent is equipped with a Tool that triggers a Lambda function.
4. The Lambda function calls a Bedrock Flow which does the following:
   1. Entity Extraction: These can be people, companies, products, etc.
   2. Gather background information: This uses the Bedrock Knowledge Base we created in the setup phase. If any entity has low confidence scores, i.e. not mentioned anywhere in the Knowledge Base it is discarded.
5. The Lambda then returns the research to the Research agent, which returns it to the Interface Supervisor agent.
6. Once additional context has been provided by the Research agent, the Interface Supervisor agent sends the research and the facts to the Article Generation agent. This agent is part of a reflection pattern we covered earlier (Lab 5):
   1. News Generation agent: This writes the main news article based on the information provided by the Research agent.
   2. Article Reviewer agent: This provides feedback to the News Generation agent and together, these agents iteratively improve the quality of the generated article.
7. The remainder of the architecture is shown for completeness, and won't be part of this lab. Feel free to implement that if you have time at the end.

Please note that this is a simplified architecture to demonstrate multi-agent collaboration, a complete architecture would incorporate storing outputs at every stage for monitoring agents, and more opportunity for human-in-the-loop capability.

Let's get started!

## Amazon Bedrock

Amazon Bedrock Agents manages the collaboration, communication, and task delegation behind the scenes. By enabling agents to work together, you can achieve higher task success rates, accuracy, and enhanced productivity. In internal benchmark testing, multi-agent collaboration has shown marked improvements compared to single-agent systems for handling complex, multi-step tasks.

Highlights of multi-agent collaboration in Amazon Bedrock
A key challenge in building eﬀective multi-agent collaboration systems is managing the complexity and overhead of coordinating multiple specialized agents at scale. Amazon Bedrock simplifies the process of building, deploying, and orchestrating effective multi-agent collaboration systems while addressing efficiency challenges through several key features and optimizations:

- __Quick setup__ – Create, deploy, and manage AI agents working together in minutes without the need for complex coding.
- __Composability__ – Integrate your existing agents as subagents within a larger agent system, allowing them to seamlessly work together to tackle complex workflows.
- __Efficient inter-agent communication__ – The supervisor agent can interact with subagents using a consistent interface, supporting parallel communication for more efficient task completion.
- __Optimized collaboration modes__ – Choose between supervisor mode and supervisor with routing mode. With routing mode, the supervisor agent will route simple requests directly to specialized subagents, bypassing full orchestration. For complex queries or when no clear intention is detected, it automatically falls back to the full supervisor mode, where the supervisor agent analyzes, breaks down problems, and coordinates multiple subagents as needed.
 - __Integrated trace and debug console__ – Visualize and analyze multi-agent interactions behind the scenes using the integrated trace and debug console.

These features collectively improve coordination capabilities, communication speed, and overall effectiveness of the multi-agent collaboration framework in tackling complex, real-world problems.

## Create a News Research workflow using Amazon Bedrock Agents
In this section we declare global variables that will act as helpers during the entire notebook.
Here's a diagram that highlights the parts of the Research agent which we are going to build:

<img src="../../imgs/lab7-architecture-diagram-research-agent.png" width="800">

Note that the Lambda function was created as part of the infrastructure setup of this workshop. Feel free to have a look at it in the Lambda section of the AWS Console.

First we restore the variables from the previous notebook:

In [1]:
%store -r

In [2]:
import boto3
import sys
import time
import uuid

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)


## Importing helper functions
In following section, we're adding bedrock_agent_helper.py on Python path, so the files can be recognized and their functionalities invoked.

In general, the helper functions handle common tasks including agent creation.

In [3]:
sys.path.insert(0, ".")
sys.path.insert(1, "..")
sys.path.insert(2, "../..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)

agents = AgentsForAmazonBedrock()

## Creating the Research agent

Let's create the Research agent that uses a Lambda tool to get research information from a Bedrock Flow.

The role of this agent is pass the facts to the Lambda function, which then passes the facts to a Bedrock Flow. The Bedrock Flow will extract entities, verify, and will return research information about the entities

We'll now Create a Bedrock Agent using the agent helper function

In [4]:
agent_description = "An agent that gathers research on facts collected regarding a news event"

# This is the instruction we pass to our research agent
agent_instruction = """You are an AI assistant designed to execute the action_group_research tool and return its exact outputs without any modifications.

When a user provides information you must execute the action_group_research tool, follow these exact steps:
1. Execute the requested tool call with the parameters provided by the user
2. Do not add any introduction, explanation, commentary, or conclusion
3. Do not modify, summarize, format, or interpret the tool's response in any way
4. Do not add your own thoughts or analysis about the tool's output
5. If the tool returns an error, return only that exact error message
6. Skip the preamble

Your sole purpose is to serve as a direct relay between the user and the tool. The user values receiving the complete, unaltered output exactly as returned by the tool.

Even if the tool output seems incomplete, confusing, or could benefit from explanation, do not add anything to it. The user specifically wants only the raw tool output for their own purposes."""

# Let's declare the foundation model the agent will be using
agent_foundation_model = [
    'us.amazon.nova-premier-v1:0'
]

In [5]:
agent_suffix = str(uuid.uuid4())[:5]
research_agent_name = f"lab-7-research-agent-{agent_suffix}"

research_agent = agents.create_agent(
    research_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False
)

research_agent_id = research_agent[0]
research_agent_alias_id = research_agent[1]
research_agent_alias_arn = research_agent[2]
agent_resourceRoleArn = research_agent[3]

research_agent

('FUCNCDY0LL',
 'TSTALIASID',
 'arn:aws:bedrock:us-east-1:188001062285:agent-alias/FUCNCDY0LL/TSTALIASID',
 'arn:aws:iam::188001062285:role/DEFAULT_AgentExecutionRole')

## Define agent Tools
In the context of Bedrock agents, tools are organized as Action Groups. An Action Group defines actions that the agent can perform. For example, you could define an Action Group called `Get background research` that helps gather background research on entities that are provided to it. If the entities don't exist in the database, they will simply be ignored.

You create an Action Broup by performing the following steps:

1. Define the parameters and information that the agent must elicit from the user for each action in the Action Group to be carried out.
2. Decide how the agent handles the parameters and information that it receives from the user and where it sends the information it elicits from the user.

To learn more about Action Groups, please refer to this [link](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html).

The function details consist of a list of parameters, defined by their name, data type (for a list of supported data types, see ParameterDetail), and whether they are required. The agent uses these configurations to determine what information it needs to elicit from the user. You can define the function detail in a JSON file with name, description, parameters, or provide a file in the OpenAPI compatible format.

To fulfill the task, you can define a Lambda function to program the business logic for an Action Group. After an Amazon Bedrock agent determines the API operation that it needs to invoke in an Action Group, it sends information from the API schema alongside relevant metadata as an input event to the Lambda function. To write your function, you must understand the following components of the Lambda function:

Input event – Contains relevant metadata and populated fields from the request body of the API operation or the function parameters for the action that the agent determines must be called.

Response – Contains relevant metadata and populated fields for the response body returned from the API operation or the function.

For more information please refer to this [link](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-lambda.html)

If a Lambda function is not feasible, another option is to choose to return control to the agent developer by sending the information in the InvokeAgent response. For more information please refer to this [link](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-returncontrol.html)

In this lab, we'll define a an Action Group with a Lambda function that simulates an API call. The API returns the research material on news entities, which are obtained from the knowledge bases that the agent can access.

In [6]:
action_group_name = f"action_group_research"
action_group_descr = "Get background research about news entities"

function_defs = [
    {
        "name": "get_research_information",
        "description": "This function calls a flow to get research information.",
        "parameters": {},
        "requireConfirmation": "DISABLED"
    }
]

Create an Action Group and connect it to the Lambda function that has already been written.

Feel free to go to the Lambda console to look at the code of the `lab7-lambda-CallFlowLambda` function

In [7]:
action_group_arn = agents.add_action_group_with_lambda(research_agent_name,
                                        call_flow_lambda_name, call_flow_lambda_arn,
                                        function_defs, action_group_name, action_group_descr)

Waiting for agent status to change. Current status CREATING
Agent id FUCNCDY0LL current status: NOT_PREPARED


We update the agent with new default temperature, topP, topK, and maximumLength values

Updating an agent requires setting a lot of values, in the cell below we grab the default values to feed them back into the `update_agent` function.

Read more about `update_agent` [here](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/update_agent.html) and about Advanced Prompts [here](https://docs.aws.amazon.com/bedrock/latest/userguide/advanced-prompts.html)

In [8]:
bedrock_agent_client = boto3.client('bedrock-agent')


# Helper function to find the right basepromptTemplate
def find_by_key_value_next(items, key, value):
    return next((item for item in items if item[key] == value), None)


# This helps us grab the correct basePromptTemplate used for the orchestration step
def get_base_prompt_template(promptType, agentId):
    # get all the info in the agent at the current state
    agent_info = bedrock_agent_client.get_agent(agentId=agentId)

    # Go through the results to find the info we need for update agent
    # You can see the full response of get_agent here:
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/get_agent.html
    prompt_orchestrations = agent_info['agent']['promptOverrideConfiguration']['promptConfigurations']
    return find_by_key_value_next(prompt_orchestrations,
                                  'promptType',
                                  promptType)['basePromptTemplate']

# We have to change the inference values only for the orchestration step
prompt_template = get_base_prompt_template('ORCHESTRATION',
                                           research_agent_id)


response = bedrock_agent_client.update_agent(
    agentId=research_agent_id,
    agentName=research_agent_name,
    description=agent_description,
    agentResourceRoleArn=agent_resourceRoleArn,
    foundationModel=agent_foundation_model[0],
    instruction=agent_instruction,
    promptOverrideConfiguration={
        "promptConfigurations": [
            {
                "inferenceConfiguration":  {
                    "temperature": 0.0,
                    "topP": 1.0,
                    "topK": 100,
                    "maximumLength": 4096
                },
                'parserMode': 'DEFAULT',
                'promptCreationMode': 'OVERRIDDEN',
                'promptType': 'ORCHESTRATION',
                'promptState': 'ENABLED',
                'basePromptTemplate': prompt_template
            }
        ]
    }
)

# An agent always has to be prepared after changes
bedrock_agent_client.prepare_agent(agentId=research_agent_id)

# Sleep to let the agent preparation finish
time.sleep(20)

## Testing the Agent
With all the components in place, let's test out our agent. We'll be using the following mock news facts that have been gathered at a news event.

In [9]:
%%time

news_facts = """NeuraHealth Solutions announced its new medical diagnostic platform called "MediScan" at their annual developer conference yesterday.
The system demonstrated 94% accuracy in early disease detection across a trial of 12,000 patients.
Dr. Eliza Chen, Chief Medical Officer at NeuraHealth, revealed the system was trained on 50 million anonymized patient records.
NeuraHealth CEO Marcus Williams stated the company invested $450 million in research and development over three years.
The platform will be piloted at five major hospital networks starting next month.
Senior Vice President of Product Development, Raj Patel, confirmed that FDA approval is expected by the third quarter.
Initial focus areas include cardiovascular disease, diabetes, and early cancer detection."""

response = agents.invoke(
        news_facts,
        research_agent[0], enable_trace=True,
        trace_level="all"
)

invokeAgent API response object: {'ResponseMetadata': {'RequestId': '2cbcb2a3-b96f-44de-bb6b-d3791825d43f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 05 Jun 2025 20:38:44 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '2cbcb2a3-b96f-44de-bb6b-d3791825d43f', 'x-amz-bedrock-agent-session-id': '9200bdc4-c861-4dc7-a87c-31da9df66fb6', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '9200bdc4-c861-4dc7-a87c-31da9df66fb6', 'completion': <botocore.eventstream.EventStream object at 0x7f9e54cd9670>}
---
{
  "agentAliasId": "TSTALIASID",
  "agentId": "FUCNCDY0LL",
  "agentVersion": "DRAFT",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:188001062285:agent-alias/FUCNCDY0LL/TSTALIASID"
    }
  ],
  "eventTime": "2025-06-05 20:38:44.379408+00:00",
  "sessionId": "9200bdc4-c861-4dc7-a87c-31da9

Here's the response from the Research Agent:

In [10]:
print(response)

## Researched Entities

- **entity_id: 1**
  **text:** NeuraHealth Solutions
  **type:** ORGANIZATION
  **subtype:** Corporation
  **confidence:** 0.95
  **research:** 
    - NeuraHealth Solutions is a leading healthcare technology company specializing in AI-powered diagnostic and healthcare decision support systems. - The company was founded in 2014 and is headquartered in San Francisco, California. - NeuraHealth Solutions went public in June 2021 with an IPO of $780 million. - The company has a market cap of $12.8 billion and reported revenue of $870 million in 2023. - NeuraHealth Solutions has a strong presence in media, with 12,485 mentions and a positive sentiment of 62% in the monitoring period from April 2023 to April 2024.
- **entity_id: 2**
  **text:** MediScan
  **type:** PRODUCT
  **subtype:** Medical Diagnostic Platform
  **confidence:** 0.95
  **research:** 
    - MediScan is an AI-powered diagnostic platform developed by NeuraHealth Solutions. - The platform uses machine 

## Create an Alias
We've just completed a test query submitted to the Research agent using it's default Alias.
The default Alias is a quick way to test an agent before integrating it into your application.
When creating a multi-agent collaboration, it's required to create an Alias explicitly so that it can be used by other agents. This is to ensure the agent is tested and validated the functionality as expected. Read more about aliases and version in the [deploying agents](https://docs.aws.amazon.com/bedrock/latest/userguide/deploy-agent.html) section of our documentation.
Since we've tested and validated our agent, let's now create an Alias for it:

In [11]:
research_agent_alias_id, research_agent_alias_arn = agents.create_agent_alias(
    research_agent_id, 'v1')

## Saving information
Let's store the environment variables to be used in other notebooks

In [12]:
%store call_flow_lambda_arn
%store research_agent_name
%store research_agent_id
%store research_agent_alias_id
%store research_agent_alias_arn

Stored 'call_flow_lambda_arn' (str)
Stored 'research_agent_name' (str)
Stored 'research_agent_id' (str)
Stored 'research_agent_alias_id' (str)
Stored 'research_agent_alias_arn' (str)


# Next step
So far, we have created a Research agent responsible for providing research material about entities it extracted from news facts. In the next notebook, we'll build an agent that generates an article based on the news facts and contextual research provided by the Research agent.